In [1]:
:opt no-lint

# CESK 기계
CESK 기계는 CEK 기계에 메모리 주소 개념을 추가해서 공유된 값 등을 표현할 수 있따.

CEK 기계는 80년대에 Friedman과 Felleisen이 SECD 기계(Landin 1964)를 더 말끔하게 정리한 것이다.
###### 참고자료
* https://www.cs.bham.ac.uk/~hxt/2015/compilers/compiling-functional.pdf
* https://en.wikipedia.org/wiki/SECD_machine
* https://www.brics.dk/RS/03/33/
* https://pages.cpsc.ucalgary.ca/~robin/FMCS/FMCS2014/Prashant_talk.pdf
* https://github.com/kseo/secd

$ x \in \textit{Nm} $

$ n \in \textit{Int} $

$ e \in \textit{Exp} ~::=~ x ~\mid~ (e_1~e_2) ~\mid~ (\lambda x.e) ~\mid~ n $

In [2]:
type Nm = String

data Exp = Var Nm      -- x
         | App Exp Exp -- e1 e2
         | Lam Nm Exp  -- \x.e
         | Lit Int     -- n
         deriving Show

$\textit{CESK} ~=~ \textit{Code} \times \textit{Env} \times \textit{Store} \times \textit{Kont}$

CEK 기계는 코드(Code), 환경(Env), 저장공간(Store), 남은일(Kont)의 네순서쌍으로 나타나며 그 네순서쌍의 각 요소는 아래와 같이 정의되는 집합의 원소이다.

 * 코드(Code)는 식(Exp)과 주소(Loc)의 합집합이다.
 
    $ \textit{Code} ~=~ \textit{Exp} \cup \textit{Loc} $

 * 환경(Env)은 이름(Nm)에서 주소(Loc)로의 유한한 대응이다.

    $ \textit{Env} ~=~ \textit{Nm} \xrightarrow{\text{fin}} \textit{Loc} $

 * 저장공간(Store)는 주소(Loc)에서 값(Value)으로의 유한한 대응이다.
 
    $ \textit{Env} ~=~ \textit{Loc} \xrightarrow{\text{fin}} \textit{Value} $
    
 * 남은일이란 프레임(Frame)의 목록이다.
 
    $ \textit{Kont} ~=~ \textit{Frame}^{*} $


값(Value)은 정수와 닫힘의 두 종류 값이 있다.

$ \textit{Value} ~=~ \textit{Int} \;\cup\; Closure $

$ v \in \textit{Value} ~::=~ n ~\mid~ \langle\lambda x.e,\sigma\rangle$

이 때 닫힘(Closure)이란 어떤 식(Exp)을 그 자유변수를 모두 포함하는 환경(Env)과 함께 엮어 놓은 것이다.

$ \textit{Closure} ~=~ \textit{Exp} \times \textit{Env}$


프레임(Frame)이란 지금 하고 있는 계산의 값이 나오면 그것으로 바로 다음에 할 일을 나타내는데, 아래와 같이 두 종류가 있다.
 * $(\ell~\bigcirc) \qquad$ 
   계산의 값이 나오면 인자로 $\ell$ 주소의 함수에 넘긴다. 즉, $\ell$ 주소에 저장된 값을 함수로써 계산되어 나오는 값에 적용한다.
 * $(\bigcirc~e~\sigma) ~\,\quad$
   계산의 값이 나오면 그 값을 함수로써 $e$에 적용한다. 물론 그러려면 $e$를 환경 $\sigma$에서 값으로 계산하기도 해야 할 것이다.



In [3]:
type CESK = (Code,Env,Store,Kont)

data Code = OpE Exp
          | OpV Loc
          deriving Show
type Env  = [(Nm,Loc)]
type Store  = [(Loc,Value)]
type Kont = [Frame]

type Loc = Int

data Value = Vint Int      -- n
           | Clos Exp Env  -- < \x.e, env >
           deriving Show
           
data Frame = FrV Loc   -- (l O)
           | FrE Exp Env -- (O e env)
           deriving Show

$ \big\langle x ~\big|~ \sigma ~\big|~ \rho ~\big|~ \kappa \big\rangle
  \longrightarrow
  \big\langle \sigma(x) ~\big|~ \sigma ~\big|~ \rho ~\big| \kappa \big\rangle $

$ \big\langle e_1~e_2 ~\big|~ \sigma ~\big|~ \rho ~\big|~ \kappa \big\rangle
  \longrightarrow
  \big\langle e_1 ~\big|~ \sigma ~\big|~ \rho ~\big|~ (\bigcirc\,e_2\;\sigma),\kappa \big\rangle $
  
$ \big\langle \lambda x.e ~\big|~ \sigma ~\big|~ \rho ~\big|~ \kappa \big\rangle
  \longrightarrow 
  \big\langle \ell ~\big|~ \sigma ~\big|~ [\ell\mapsto \langle\lambda x.e,\sigma\rangle]\rho ~\big|~ \kappa \big\rangle \quad (\ell \notin \mathrm{dom}(\rho))$
  
$ \big\langle \ell_1 ~\big|~ \sigma_1 ~\big|~ \rho ~\big|~ (\bigcirc\,e_2\;\sigma_2),\kappa  \big\rangle
  \longrightarrow
  \big\langle e_2 ~\big|~ \sigma_2 ~\big|~ \rho ~\big|~ (\ell_1\,\bigcirc),\kappa  \big\rangle $

$ \big\langle \ell_2 ~\big|~ \sigma_2 ~\big|~ \rho ~\big|~ (\ell_1\,\bigcirc),\kappa  \big\rangle
  \longrightarrow
  \big\langle e ~\big|~ [x\mapsto \ell_2]\sigma_1 ~\big|~ \rho ~\big|~ \kappa \big\rangle \quad (\rho(\ell_1) = \langle\lambda x.e,\sigma_1\rangle)$

In [4]:
lookup' x env = case lookup x env of Just v  -> v
                                     Nothing -> error "x is unknown"

newLoc rho = maximum (-1 : dom rho) + 1 where dom = map fst

In [5]:
newLoc [(0,Vint 3),(7,Vint 4),(4,Vint 5)]

8

In [6]:
step :: CESK -> Maybe CESK
step (OpE(Var x),     env, rho, k) = Just (OpV(lookup' x env), env, rho, k)
step (OpE(App e1 e2), env, rho, k) = Just (OpE e1, env, rho, FrE e2 env : k)
step (OpE e@Lam{},    env, rho, k) = Just (OpV l, env, (l,Clos e env):rho, k)
    where l = newLoc rho
step (OpE(Lit n),     env, rho, k) = Just (OpV l, env, (l,Vint n):rho, k)
    where l = newLoc rho
step (OpV l1, env1, rho, FrE e2 env2 : k) = Just (OpE e2, env2, rho, FrV l1 : k)
step (OpV l2, env2, rho, FrV l1 : k) = Just (OpE e, (x,l2) : env1, rho, k)
    where Clos (Lam x e) env1 = lookup' l1 rho
step _ = Nothing

$\underline{10}$

In [8]:
{-# LANGUAGE FlexibleInstances #-}
import IHaskell.Display
import Data.List (intersperse, intercalate)

class TeX a where
  toTeX :: a -> String

instance TeX Exp where
  toTeX (Var x) = x
  toTeX (App e1 e2) = "("++toTeX e1++"\\;"++toTeX e2++")" 
  toTeX (Lam x e) = "(\\lambda{}"++x++"."++toTeX e++")"
  toTeX (Lit n) = show n

instance TeX Code where
  toTeX (OpE e) = toTeX e 
  toTeX (OpV l) = showLoc l

showLoc l = "\\underline{"++show l++"}"

instance TeX Value where
  toTeX (Vint n) = show n
  toTeX (Clos e env) = "\\langle{}" ++ toTeX e ++ "," ++ toTeX env ++ "\\rangle{}"

instance TeX Frame where
  toTeX (FrV l) = "("++showLoc l++"\\;\\bigcirc{})"
  toTeX (FrE e env) = "(\\bigcirc{}\\;"++toTeX e++"\\;"++toTeX env++")"

instance TeX (Nm,Loc) where
  toTeX (x,l) = x++"\\mapsto{}"++showLoc l

instance TeX (Loc,Value) where
  toTeX (l,v) = showLoc l++"\\mapsto{}"++toTeX v

instance TeX CESK where
  toTeX (c,env,rho,k) = "\\big\\langle{}"
                 ++ toTeX c
                 ++ "\\;\\big|\\;" ++ toTeX env
                 ++ "\\;\\big|\\;" ++ toTeX rho
                 ++ "\\;\\big|\\;" ++ toTeX k
                 ++ "\\big\\rangle{}"

instance TeX a => TeX (Maybe a) where
  toTeX (Just x)= "\\texttt{Just}("++toTeX x++")"
  toTeX Nothing = "\\texttt{Nothing}"

instance TeX a => TeX [a] where
  toTeX xs = "[" ++ (intercalate "," $ map toTeX xs) ++ "]"

-- revTeX = toTeX . reverse

newtype LaTeX a = LaTeX a

htmlTeX a = html $ "$"++toTeX a++"$"

instance TeX a => IHaskellDisplay (LaTeX a) where
  display (LaTeX a) = return $ Display [htmlTeX a]

In [9]:
fst_1_2 = App (App (Lam "x" $ Lam "y" $ Var "x") (Lit 1)) (Lit 2)
fst_1_2
LaTeX fst_1_2

App (App (Lam "x" (Lam "y" (Var "x"))) (Lit 1)) (Lit 2)

$(((\lambda{}x.(\lambda{}y.x))\;1)\;2)$

In [10]:
step (OpE fst_1_2, [], [], [])

Just (OpE (App (Lam "x" (Lam "y" (Var "x"))) (Lit 1)),[],[],[FrE (Lit 2) []])

In [11]:
step (OpV 0, [], [(0,Vint 1)], [])

Nothing

In [12]:
Just(OpE fst_1_2, [], [(9,(Vint 3))], []) :: Maybe CESK
case it of Just cek -> step cek
           Nothing  -> Nothing
case it of Just cek -> step cek
           Nothing  -> Nothing
case it of Just cek -> step cek
           Nothing  -> Nothing
case it of Just cek -> step cek
           Nothing  -> Nothing
case it of Just cek -> step cek
           Nothing  -> Nothing
case it of Just cek -> step cek
           Nothing  -> Nothing
case it of Just cek -> step cek
           Nothing  -> Nothing
case it of Just cek -> step cek
           Nothing  -> Nothing
case it of Just cek -> step cek
           Nothing  -> Nothing
case it of Just cek -> step cek
           Nothing  -> Nothing
case it of Just cek -> step cek
           Nothing  -> Nothing
case it of Just cek -> step cek
           Nothing  -> Nothing
case it of Just cek -> step cek
           Nothing  -> Nothing

Just (OpE (App (App (Lam "x" (Lam "y" (Var "x"))) (Lit 1)) (Lit 2)),[],[(9,Vint 3)],[])

Just (OpE (App (Lam "x" (Lam "y" (Var "x"))) (Lit 1)),[],[(9,Vint 3)],[FrE (Lit 2) []])

Just (OpE (Lam "x" (Lam "y" (Var "x"))),[],[(9,Vint 3)],[FrE (Lit 1) [],FrE (Lit 2) []])

Just (OpV 10,[],[(10,Clos (Lam "x" (Lam "y" (Var "x"))) []),(9,Vint 3)],[FrE (Lit 1) [],FrE (Lit 2) []])

Just (OpE (Lit 1),[],[(10,Clos (Lam "x" (Lam "y" (Var "x"))) []),(9,Vint 3)],[FrV 10,FrE (Lit 2) []])

...

Just (OpV 13,[],[(13,Vint 2),(12,Clos (Lam "y" (Var "x")) [("x",11)]),(11,Vint 1),(10,Clos (Lam "x" (Lam "y" (Var "x"))) []),(9,Vint 3)],[FrV 12])

Just (OpE (Var "x"),[("y",13),("x",11)],[(13,Vint 2),(12,Clos (Lam "y" (Var "x")) [("x",11)]),(11,Vint 1),(10,Clos (Lam "x" (Lam "y" (Var "x"))) []),(9,Vint 3)],[])

Just (OpV 11,[("y",13),("x",11)],[(13,Vint 2),(12,Clos (Lam "y" (Var "x")) [("x",11)]),(11,Vint 1),(10,Clos (Lam "x" (Lam "y" (Var "x"))) []),(9,Vint 3)],[])

Nothing

Nothing

In [13]:
Just(OpE fst_1_2, [], [(9,(Vint 3))], []) :: Maybe CESK
step =<< it
step =<< it
step =<< it
step =<< it
step =<< it
step =<< it
step =<< it
step =<< it
step =<< it
step =<< it
step =<< it
step =<< it
step =<< it

Just (OpE (App (App (Lam "x" (Lam "y" (Var "x"))) (Lit 1)) (Lit 2)),[],[(9,Vint 3)],[])

Just (OpE (App (Lam "x" (Lam "y" (Var "x"))) (Lit 1)),[],[(9,Vint 3)],[FrE (Lit 2) []])

Just (OpE (Lam "x" (Lam "y" (Var "x"))),[],[(9,Vint 3)],[FrE (Lit 1) [],FrE (Lit 2) []])

Just (OpV 10,[],[(10,Clos (Lam "x" (Lam "y" (Var "x"))) []),(9,Vint 3)],[FrE (Lit 1) [],FrE (Lit 2) []])

Just (OpE (Lit 1),[],[(10,Clos (Lam "x" (Lam "y" (Var "x"))) []),(9,Vint 3)],[FrV 10,FrE (Lit 2) []])

...

Just (OpV 13,[],[(13,Vint 2),(12,Clos (Lam "y" (Var "x")) [("x",11)]),(11,Vint 1),(10,Clos (Lam "x" (Lam "y" (Var "x"))) []),(9,Vint 3)],[FrV 12])

Just (OpE (Var "x"),[("y",13),("x",11)],[(13,Vint 2),(12,Clos (Lam "y" (Var "x")) [("x",11)]),(11,Vint 1),(10,Clos (Lam "x" (Lam "y" (Var "x"))) []),(9,Vint 3)],[])

Just (OpV 11,[("y",13),("x",11)],[(13,Vint 2),(12,Clos (Lam "y" (Var "x")) [("x",11)]),(11,Vint 1),(10,Clos (Lam "x" (Lam "y" (Var "x"))) []),(9,Vint 3)],[])

Nothing

Nothing

In [14]:
m0  =  Just(OpE fst_1_2, [], [(9,(Vint 3))], []) :: Maybe CESK
m1  =  step =<< m0
m2  =  step =<< m1
m3  =  step =<< m2
m4  =  step =<< m3
m5  =  step =<< m4
m6  =  step =<< m5
m7  =  step =<< m6
m8  =  step =<< m7
m9  =  step =<< m8
m10 =  step =<< m9
m11 =  step =<< m10
m12 =  step =<< m11
m13 =  step =<< m12

In [15]:
m0
m1
m2
m3
m4
m5
m6
m7
m8
m9
m10
m11
m12
m13

Just (OpE (App (App (Lam "x" (Lam "y" (Var "x"))) (Lit 1)) (Lit 2)),[],[(9,Vint 3)],[])

Just (OpE (App (Lam "x" (Lam "y" (Var "x"))) (Lit 1)),[],[(9,Vint 3)],[FrE (Lit 2) []])

Just (OpE (Lam "x" (Lam "y" (Var "x"))),[],[(9,Vint 3)],[FrE (Lit 1) [],FrE (Lit 2) []])

Just (OpV 10,[],[(10,Clos (Lam "x" (Lam "y" (Var "x"))) []),(9,Vint 3)],[FrE (Lit 1) [],FrE (Lit 2) []])

Just (OpE (Lit 1),[],[(10,Clos (Lam "x" (Lam "y" (Var "x"))) []),(9,Vint 3)],[FrV 10,FrE (Lit 2) []])

...

Just (OpV 13,[],[(13,Vint 2),(12,Clos (Lam "y" (Var "x")) [("x",11)]),(11,Vint 1),(10,Clos (Lam "x" (Lam "y" (Var "x"))) []),(9,Vint 3)],[FrV 12])

Just (OpE (Var "x"),[("y",13),("x",11)],[(13,Vint 2),(12,Clos (Lam "y" (Var "x")) [("x",11)]),(11,Vint 1),(10,Clos (Lam "x" (Lam "y" (Var "x"))) []),(9,Vint 3)],[])

Just (OpV 11,[("y",13),("x",11)],[(13,Vint 2),(12,Clos (Lam "y" (Var "x")) [("x",11)]),(11,Vint 1),(10,Clos (Lam "x" (Lam "y" (Var "x"))) []),(9,Vint 3)],[])

Nothing

Nothing

In [16]:
print m0
print m1
print m2
print m3
print m4
print m5
print m6
print m7
print m8
print m9
print m10
print m11
print m12
print m13

Just (OpE (App (App (Lam "x" (Lam "y" (Var "x"))) (Lit 1)) (Lit 2)),[],[(9,Vint 3)],[])

Just (OpE (App (Lam "x" (Lam "y" (Var "x"))) (Lit 1)),[],[(9,Vint 3)],[FrE (Lit 2) []])

Just (OpE (Lam "x" (Lam "y" (Var "x"))),[],[(9,Vint 3)],[FrE (Lit 1) [],FrE (Lit 2) []])

Just (OpV 10,[],[(10,Clos (Lam "x" (Lam "y" (Var "x"))) []),(9,Vint 3)],[FrE (Lit 1) [],FrE (Lit 2) []])

Just (OpE (Lit 1),[],[(10,Clos (Lam "x" (Lam "y" (Var "x"))) []),(9,Vint 3)],[FrV 10,FrE (Lit 2) []])

...

Just (OpV 13,[],[(13,Vint 2),(12,Clos (Lam "y" (Var "x")) [("x",11)]),(11,Vint 1),(10,Clos (Lam "x" (Lam "y" (Var "x"))) []),(9,Vint 3)],[FrV 12])

Just (OpE (Var "x"),[("y",13),("x",11)],[(13,Vint 2),(12,Clos (Lam "y" (Var "x")) [("x",11)]),(11,Vint 1),(10,Clos (Lam "x" (Lam "y" (Var "x"))) []),(9,Vint 3)],[])

Just (OpV 11,[("y",13),("x",11)],[(13,Vint 2),(12,Clos (Lam "y" (Var "x")) [("x",11)]),(11,Vint 1),(10,Clos (Lam "x" (Lam "y" (Var "x"))) []),(9,Vint 3)],[])

Nothing

Nothing

In [17]:
LaTeX m0
LaTeX m1
LaTeX m2
LaTeX m3
LaTeX m4
LaTeX m5
LaTeX m6
LaTeX m7
LaTeX m8
LaTeX m9
LaTeX m10
LaTeX m11
LaTeX m12
LaTeX m13

$\texttt{Just}(\big\langle{}(((\lambda{}x.(\lambda{}y.x))\;1)\;2)\;\big|\;[]\;\big|\;[\underline{9}\mapsto{}3]\;\big|\;[]\big\rangle{})$

$\texttt{Just}(\big\langle{}((\lambda{}x.(\lambda{}y.x))\;1)\;\big|\;[]\;\big|\;[\underline{9}\mapsto{}3]\;\big|\;[(\bigcirc{}\;2\;[])]\big\rangle{})$

$\texttt{Just}(\big\langle{}(\lambda{}x.(\lambda{}y.x))\;\big|\;[]\;\big|\;[\underline{9}\mapsto{}3]\;\big|\;[(\bigcirc{}\;1\;[]),(\bigcirc{}\;2\;[])]\big\rangle{})$

$\texttt{Just}(\big\langle{}\underline{10}\;\big|\;[]\;\big|\;[\underline{10}\mapsto{}\langle{}(\lambda{}x.(\lambda{}y.x)),[]\rangle{},\underline{9}\mapsto{}3]\;\big|\;[(\bigcirc{}\;1\;[]),(\bigcirc{}\;2\;[])]\big\rangle{})$

$\texttt{Just}(\big\langle{}1\;\big|\;[]\;\big|\;[\underline{10}\mapsto{}\langle{}(\lambda{}x.(\lambda{}y.x)),[]\rangle{},\underline{9}\mapsto{}3]\;\big|\;[(\underline{10}\;\bigcirc{}),(\bigcirc{}\;2\;[])]\big\rangle{})$

...

$\texttt{Just}(\big\langle{}\underline{13}\;\big|\;[]\;\big|\;[\underline{13}\mapsto{}2,\underline{12}\mapsto{}\langle{}(\lambda{}y.x),[x\mapsto{}\underline{11}]\rangle{},\underline{11}\mapsto{}1,\underline{10}\mapsto{}\langle{}(\lambda{}x.(\lambda{}y.x)),[]\rangle{},\underline{9}\mapsto{}3]\;\big|\;[(\underline{12}\;\bigcirc{})]\big\rangle{})$

$\texttt{Just}(\big\langle{}x\;\big|\;[y\mapsto{}\underline{13},x\mapsto{}\underline{11}]\;\big|\;[\underline{13}\mapsto{}2,\underline{12}\mapsto{}\langle{}(\lambda{}y.x),[x\mapsto{}\underline{11}]\rangle{},\underline{11}\mapsto{}1,\underline{10}\mapsto{}\langle{}(\lambda{}x.(\lambda{}y.x)),[]\rangle{},\underline{9}\mapsto{}3]\;\big|\;[]\big\rangle{})$

$\texttt{Just}(\big\langle{}\underline{11}\;\big|\;[y\mapsto{}\underline{13},x\mapsto{}\underline{11}]\;\big|\;[\underline{13}\mapsto{}2,\underline{12}\mapsto{}\langle{}(\lambda{}y.x),[x\mapsto{}\underline{11}]\rangle{},\underline{11}\mapsto{}1,\underline{10}\mapsto{}\langle{}(\lambda{}x.(\lambda{}y.x)),[]\rangle{},\underline{9}\mapsto{}3]\;\big|\;[]\big\rangle{})$

$\texttt{Nothing}$

$\texttt{Nothing}$

In [18]:
mapM_ print [m0,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,m11,m12,m13]

Just (OpE (App (App (Lam "x" (Lam "y" (Var "x"))) (Lit 1)) (Lit 2)),[],[(9,Vint 3)],[])
Just (OpE (App (Lam "x" (Lam "y" (Var "x"))) (Lit 1)),[],[(9,Vint 3)],[FrE (Lit 2) []])
Just (OpE (Lam "x" (Lam "y" (Var "x"))),[],[(9,Vint 3)],[FrE (Lit 1) [],FrE (Lit 2) []])
Just (OpV 10,[],[(10,Clos (Lam "x" (Lam "y" (Var "x"))) []),(9,Vint 3)],[FrE (Lit 1) [],FrE (Lit 2) []])
Just (OpE (Lit 1),[],[(10,Clos (Lam "x" (Lam "y" (Var "x"))) []),(9,Vint 3)],[FrV 10,FrE (Lit 2) []])
Just (OpV 11,[],[(11,Vint 1),(10,Clos (Lam "x" (Lam "y" (Var "x"))) []),(9,Vint 3)],[FrV 10,FrE (Lit 2) []])
Just (OpE (Lam "y" (Var "x")),[("x",11)],[(11,Vint 1),(10,Clos (Lam "x" (Lam "y" (Var "x"))) []),(9,Vint 3)],[FrE (Lit 2) []])
Just (OpV 12,[("x",11)],[(12,Clos (Lam "y" (Var "x")) [("x",11)]),(11,Vint 1),(10,Clos (Lam "x" (Lam "y" (Var "x"))) []),(9,Vint 3)],[FrE (Lit 2) []])
Just (OpE (Lit 2),[],[(12,Clos (Lam "y" (Var "x")) [("x",11)]),(11,Vint 1),(10,Clos (Lam "x" (Lam "y" (Var "x"))) []),(9,Vint 3)],[FrV 12])
J

In [19]:
map LaTeX [m0,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,m11,m12,m13]

$\texttt{Just}(\big\langle{}(((\lambda{}x.(\lambda{}y.x))\;1)\;2)\;\big|\;[]\;\big|\;[\underline{9}\mapsto{}3]\;\big|\;[]\big\rangle{})$

$\texttt{Just}(\big\langle{}((\lambda{}x.(\lambda{}y.x))\;1)\;\big|\;[]\;\big|\;[\underline{9}\mapsto{}3]\;\big|\;[(\bigcirc{}\;2\;[])]\big\rangle{})$

$\texttt{Just}(\big\langle{}(\lambda{}x.(\lambda{}y.x))\;\big|\;[]\;\big|\;[\underline{9}\mapsto{}3]\;\big|\;[(\bigcirc{}\;1\;[]),(\bigcirc{}\;2\;[])]\big\rangle{})$

$\texttt{Just}(\big\langle{}\underline{10}\;\big|\;[]\;\big|\;[\underline{10}\mapsto{}\langle{}(\lambda{}x.(\lambda{}y.x)),[]\rangle{},\underline{9}\mapsto{}3]\;\big|\;[(\bigcirc{}\;1\;[]),(\bigcirc{}\;2\;[])]\big\rangle{})$

$\texttt{Just}(\big\langle{}1\;\big|\;[]\;\big|\;[\underline{10}\mapsto{}\langle{}(\lambda{}x.(\lambda{}y.x)),[]\rangle{},\underline{9}\mapsto{}3]\;\big|\;[(\underline{10}\;\bigcirc{}),(\bigcirc{}\;2\;[])]\big\rangle{})$

...

$\texttt{Just}(\big\langle{}\underline{13}\;\big|\;[]\;\big|\;[\underline{13}\mapsto{}2,\underline{12}\mapsto{}\langle{}(\lambda{}y.x),[x\mapsto{}\underline{11}]\rangle{},\underline{11}\mapsto{}1,\underline{10}\mapsto{}\langle{}(\lambda{}x.(\lambda{}y.x)),[]\rangle{},\underline{9}\mapsto{}3]\;\big|\;[(\underline{12}\;\bigcirc{})]\big\rangle{})$

$\texttt{Just}(\big\langle{}x\;\big|\;[y\mapsto{}\underline{13},x\mapsto{}\underline{11}]\;\big|\;[\underline{13}\mapsto{}2,\underline{12}\mapsto{}\langle{}(\lambda{}y.x),[x\mapsto{}\underline{11}]\rangle{},\underline{11}\mapsto{}1,\underline{10}\mapsto{}\langle{}(\lambda{}x.(\lambda{}y.x)),[]\rangle{},\underline{9}\mapsto{}3]\;\big|\;[]\big\rangle{})$

$\texttt{Just}(\big\langle{}\underline{11}\;\big|\;[y\mapsto{}\underline{13},x\mapsto{}\underline{11}]\;\big|\;[\underline{13}\mapsto{}2,\underline{12}\mapsto{}\langle{}(\lambda{}y.x),[x\mapsto{}\underline{11}]\rangle{},\underline{11}\mapsto{}1,\underline{10}\mapsto{}\langle{}(\lambda{}x.(\lambda{}y.x)),[]\rangle{},\underline{9}\mapsto{}3]\;\big|\;[]\big\rangle{})$

$\texttt{Nothing}$

$\texttt{Nothing}$

In [20]:
:type iterate
:type \m -> step =<< m
:type (step =<<)
:type iterate (step =<<)
:type iterate (step =<<) m0

iterate :: forall a. (a -> a) -> a -> [a]

\m -> step =<< m :: Maybe CESK -> Maybe CESK

(step =<<) :: Maybe CESK -> Maybe CESK

iterate (step =<<) :: Maybe CESK -> [Maybe CESK]

iterate (step =<<) m0 :: [Maybe CESK]

In [21]:
take 14 $ iterate (step =<<) m0

[Just (OpE (App (App (Lam "x" (Lam "y" (Var "x"))) (Lit 1)) (Lit 2)),[],[(9,Vint 3)],[]),Just (OpE (App (Lam "x" (Lam "y" (Var "x"))) (Lit 1)),[],[(9,Vint 3)],[FrE (Lit 2) []]),Just (OpE (Lam "x" (Lam "y" (Var "x"))),[],[(9,Vint 3)],[FrE (Lit 1) [],FrE (Lit 2) []]),Just (OpV 10,[],[(10,Clos (Lam "x" (Lam "y" (Var "x"))) []),(9,Vint 3)],[FrE (Lit 1) [],FrE (Lit 2) []]),Just (OpE (Lit 1),[],[(10,Clos (Lam "x" (Lam "y" (Var "x"))) []),(9,Vint 3)],[FrV 10,FrE (Lit 2) []]),Just (OpV 11,[],[(11,Vint 1),(10,Clos (Lam "x" (Lam "y" (Var "x"))) []),(9,Vint 3)],[FrV 10,FrE (Lit 2) []]),Just (OpE (Lam "y" (Var "x")),[("x",11)],[(11,Vint 1),(10,Clos (Lam "x" (Lam "y" (Var "x"))) []),(9,Vint 3)],[FrE (Lit 2) []]),Just (OpV 12,[("x",11)],[(12,Clos (Lam "y" (Var "x")) [("x",11)]),(11,Vint 1),(10,Clos (Lam "x" (Lam "y" (Var "x"))) []),(9,Vint 3)],[FrE (Lit 2) []]),Just (OpE (Lit 2),[],[(12,Clos (Lam "y" (Var "x")) [("x",11)]),(11,Vint 1),(10,Clos (Lam "x" (Lam "y" (Var "x"))) []),(9,Vint 3)],[FrV 12]),

In [22]:
mapM_ print . take 14 $ iterate (step =<<) m0

Just (OpE (App (App (Lam "x" (Lam "y" (Var "x"))) (Lit 1)) (Lit 2)),[],[(9,Vint 3)],[])
Just (OpE (App (Lam "x" (Lam "y" (Var "x"))) (Lit 1)),[],[(9,Vint 3)],[FrE (Lit 2) []])
Just (OpE (Lam "x" (Lam "y" (Var "x"))),[],[(9,Vint 3)],[FrE (Lit 1) [],FrE (Lit 2) []])
Just (OpV 10,[],[(10,Clos (Lam "x" (Lam "y" (Var "x"))) []),(9,Vint 3)],[FrE (Lit 1) [],FrE (Lit 2) []])
Just (OpE (Lit 1),[],[(10,Clos (Lam "x" (Lam "y" (Var "x"))) []),(9,Vint 3)],[FrV 10,FrE (Lit 2) []])
Just (OpV 11,[],[(11,Vint 1),(10,Clos (Lam "x" (Lam "y" (Var "x"))) []),(9,Vint 3)],[FrV 10,FrE (Lit 2) []])
Just (OpE (Lam "y" (Var "x")),[("x",11)],[(11,Vint 1),(10,Clos (Lam "x" (Lam "y" (Var "x"))) []),(9,Vint 3)],[FrE (Lit 2) []])
Just (OpV 12,[("x",11)],[(12,Clos (Lam "y" (Var "x")) [("x",11)]),(11,Vint 1),(10,Clos (Lam "x" (Lam "y" (Var "x"))) []),(9,Vint 3)],[FrE (Lit 2) []])
Just (OpE (Lit 2),[],[(12,Clos (Lam "y" (Var "x")) [("x",11)]),(11,Vint 1),(10,Clos (Lam "x" (Lam "y" (Var "x"))) []),(9,Vint 3)],[FrV 12])
J

In [23]:
map LaTeX . take 14 $ iterate (step =<<) m0

$\texttt{Just}(\big\langle{}(((\lambda{}x.(\lambda{}y.x))\;1)\;2)\;\big|\;[]\;\big|\;[\underline{9}\mapsto{}3]\;\big|\;[]\big\rangle{})$

$\texttt{Just}(\big\langle{}((\lambda{}x.(\lambda{}y.x))\;1)\;\big|\;[]\;\big|\;[\underline{9}\mapsto{}3]\;\big|\;[(\bigcirc{}\;2\;[])]\big\rangle{})$

$\texttt{Just}(\big\langle{}(\lambda{}x.(\lambda{}y.x))\;\big|\;[]\;\big|\;[\underline{9}\mapsto{}3]\;\big|\;[(\bigcirc{}\;1\;[]),(\bigcirc{}\;2\;[])]\big\rangle{})$

$\texttt{Just}(\big\langle{}\underline{10}\;\big|\;[]\;\big|\;[\underline{10}\mapsto{}\langle{}(\lambda{}x.(\lambda{}y.x)),[]\rangle{},\underline{9}\mapsto{}3]\;\big|\;[(\bigcirc{}\;1\;[]),(\bigcirc{}\;2\;[])]\big\rangle{})$

$\texttt{Just}(\big\langle{}1\;\big|\;[]\;\big|\;[\underline{10}\mapsto{}\langle{}(\lambda{}x.(\lambda{}y.x)),[]\rangle{},\underline{9}\mapsto{}3]\;\big|\;[(\underline{10}\;\bigcirc{}),(\bigcirc{}\;2\;[])]\big\rangle{})$

...

$\texttt{Just}(\big\langle{}\underline{13}\;\big|\;[]\;\big|\;[\underline{13}\mapsto{}2,\underline{12}\mapsto{}\langle{}(\lambda{}y.x),[x\mapsto{}\underline{11}]\rangle{},\underline{11}\mapsto{}1,\underline{10}\mapsto{}\langle{}(\lambda{}x.(\lambda{}y.x)),[]\rangle{},\underline{9}\mapsto{}3]\;\big|\;[(\underline{12}\;\bigcirc{})]\big\rangle{})$

$\texttt{Just}(\big\langle{}x\;\big|\;[y\mapsto{}\underline{13},x\mapsto{}\underline{11}]\;\big|\;[\underline{13}\mapsto{}2,\underline{12}\mapsto{}\langle{}(\lambda{}y.x),[x\mapsto{}\underline{11}]\rangle{},\underline{11}\mapsto{}1,\underline{10}\mapsto{}\langle{}(\lambda{}x.(\lambda{}y.x)),[]\rangle{},\underline{9}\mapsto{}3]\;\big|\;[]\big\rangle{})$

$\texttt{Just}(\big\langle{}\underline{11}\;\big|\;[y\mapsto{}\underline{13},x\mapsto{}\underline{11}]\;\big|\;[\underline{13}\mapsto{}2,\underline{12}\mapsto{}\langle{}(\lambda{}y.x),[x\mapsto{}\underline{11}]\rangle{},\underline{11}\mapsto{}1,\underline{10}\mapsto{}\langle{}(\lambda{}x.(\lambda{}y.x)),[]\rangle{},\underline{9}\mapsto{}3]\;\big|\;[]\big\rangle{})$

$\texttt{Nothing}$

$\texttt{Nothing}$

In [24]:
isJust (Just _) = True
isJust Nothing  = False

In [25]:
takeWhile isJust $ iterate (step =<<) m0

[Just (OpE (App (App (Lam "x" (Lam "y" (Var "x"))) (Lit 1)) (Lit 2)),[],[(9,Vint 3)],[]),Just (OpE (App (Lam "x" (Lam "y" (Var "x"))) (Lit 1)),[],[(9,Vint 3)],[FrE (Lit 2) []]),Just (OpE (Lam "x" (Lam "y" (Var "x"))),[],[(9,Vint 3)],[FrE (Lit 1) [],FrE (Lit 2) []]),Just (OpV 10,[],[(10,Clos (Lam "x" (Lam "y" (Var "x"))) []),(9,Vint 3)],[FrE (Lit 1) [],FrE (Lit 2) []]),Just (OpE (Lit 1),[],[(10,Clos (Lam "x" (Lam "y" (Var "x"))) []),(9,Vint 3)],[FrV 10,FrE (Lit 2) []]),Just (OpV 11,[],[(11,Vint 1),(10,Clos (Lam "x" (Lam "y" (Var "x"))) []),(9,Vint 3)],[FrV 10,FrE (Lit 2) []]),Just (OpE (Lam "y" (Var "x")),[("x",11)],[(11,Vint 1),(10,Clos (Lam "x" (Lam "y" (Var "x"))) []),(9,Vint 3)],[FrE (Lit 2) []]),Just (OpV 12,[("x",11)],[(12,Clos (Lam "y" (Var "x")) [("x",11)]),(11,Vint 1),(10,Clos (Lam "x" (Lam "y" (Var "x"))) []),(9,Vint 3)],[FrE (Lit 2) []]),Just (OpE (Lit 2),[],[(12,Clos (Lam "y" (Var "x")) [("x",11)]),(11,Vint 1),(10,Clos (Lam "x" (Lam "y" (Var "x"))) []),(9,Vint 3)],[FrV 12]),

In [26]:
mapM_ print $ takeWhile isJust $ iterate (step =<<) m0

Just (OpE (App (App (Lam "x" (Lam "y" (Var "x"))) (Lit 1)) (Lit 2)),[],[(9,Vint 3)],[])
Just (OpE (App (Lam "x" (Lam "y" (Var "x"))) (Lit 1)),[],[(9,Vint 3)],[FrE (Lit 2) []])
Just (OpE (Lam "x" (Lam "y" (Var "x"))),[],[(9,Vint 3)],[FrE (Lit 1) [],FrE (Lit 2) []])
Just (OpV 10,[],[(10,Clos (Lam "x" (Lam "y" (Var "x"))) []),(9,Vint 3)],[FrE (Lit 1) [],FrE (Lit 2) []])
Just (OpE (Lit 1),[],[(10,Clos (Lam "x" (Lam "y" (Var "x"))) []),(9,Vint 3)],[FrV 10,FrE (Lit 2) []])
Just (OpV 11,[],[(11,Vint 1),(10,Clos (Lam "x" (Lam "y" (Var "x"))) []),(9,Vint 3)],[FrV 10,FrE (Lit 2) []])
Just (OpE (Lam "y" (Var "x")),[("x",11)],[(11,Vint 1),(10,Clos (Lam "x" (Lam "y" (Var "x"))) []),(9,Vint 3)],[FrE (Lit 2) []])
Just (OpV 12,[("x",11)],[(12,Clos (Lam "y" (Var "x")) [("x",11)]),(11,Vint 1),(10,Clos (Lam "x" (Lam "y" (Var "x"))) []),(9,Vint 3)],[FrE (Lit 2) []])
Just (OpE (Lit 2),[],[(12,Clos (Lam "y" (Var "x")) [("x",11)]),(11,Vint 1),(10,Clos (Lam "x" (Lam "y" (Var "x"))) []),(9,Vint 3)],[FrV 12])
J

In [27]:
map LaTeX . takeWhile isJust $ iterate (step =<<) m0

$\texttt{Just}(\big\langle{}(((\lambda{}x.(\lambda{}y.x))\;1)\;2)\;\big|\;[]\;\big|\;[\underline{9}\mapsto{}3]\;\big|\;[]\big\rangle{})$

$\texttt{Just}(\big\langle{}((\lambda{}x.(\lambda{}y.x))\;1)\;\big|\;[]\;\big|\;[\underline{9}\mapsto{}3]\;\big|\;[(\bigcirc{}\;2\;[])]\big\rangle{})$

$\texttt{Just}(\big\langle{}(\lambda{}x.(\lambda{}y.x))\;\big|\;[]\;\big|\;[\underline{9}\mapsto{}3]\;\big|\;[(\bigcirc{}\;1\;[]),(\bigcirc{}\;2\;[])]\big\rangle{})$

$\texttt{Just}(\big\langle{}\underline{10}\;\big|\;[]\;\big|\;[\underline{10}\mapsto{}\langle{}(\lambda{}x.(\lambda{}y.x)),[]\rangle{},\underline{9}\mapsto{}3]\;\big|\;[(\bigcirc{}\;1\;[]),(\bigcirc{}\;2\;[])]\big\rangle{})$

$\texttt{Just}(\big\langle{}1\;\big|\;[]\;\big|\;[\underline{10}\mapsto{}\langle{}(\lambda{}x.(\lambda{}y.x)),[]\rangle{},\underline{9}\mapsto{}3]\;\big|\;[(\underline{10}\;\bigcirc{}),(\bigcirc{}\;2\;[])]\big\rangle{})$

...

$\texttt{Just}(\big\langle{}\underline{12}\;\big|\;[x\mapsto{}\underline{11}]\;\big|\;[\underline{12}\mapsto{}\langle{}(\lambda{}y.x),[x\mapsto{}\underline{11}]\rangle{},\underline{11}\mapsto{}1,\underline{10}\mapsto{}\langle{}(\lambda{}x.(\lambda{}y.x)),[]\rangle{},\underline{9}\mapsto{}3]\;\big|\;[(\bigcirc{}\;2\;[])]\big\rangle{})$

$\texttt{Just}(\big\langle{}2\;\big|\;[]\;\big|\;[\underline{12}\mapsto{}\langle{}(\lambda{}y.x),[x\mapsto{}\underline{11}]\rangle{},\underline{11}\mapsto{}1,\underline{10}\mapsto{}\langle{}(\lambda{}x.(\lambda{}y.x)),[]\rangle{},\underline{9}\mapsto{}3]\;\big|\;[(\underline{12}\;\bigcirc{})]\big\rangle{})$

$\texttt{Just}(\big\langle{}\underline{13}\;\big|\;[]\;\big|\;[\underline{13}\mapsto{}2,\underline{12}\mapsto{}\langle{}(\lambda{}y.x),[x\mapsto{}\underline{11}]\rangle{},\underline{11}\mapsto{}1,\underline{10}\mapsto{}\langle{}(\lambda{}x.(\lambda{}y.x)),[]\rangle{},\underline{9}\mapsto{}3]\;\big|\;[(\underline{12}\;\bigcirc{})]\big\rangle{})$

$\texttt{Just}(\big\langle{}x\;\big|\;[y\mapsto{}\underline{13},x\mapsto{}\underline{11}]\;\big|\;[\underline{13}\mapsto{}2,\underline{12}\mapsto{}\langle{}(\lambda{}y.x),[x\mapsto{}\underline{11}]\rangle{},\underline{11}\mapsto{}1,\underline{10}\mapsto{}\langle{}(\lambda{}x.(\lambda{}y.x)),[]\rangle{},\underline{9}\mapsto{}3]\;\big|\;[]\big\rangle{})$

$\texttt{Just}(\big\langle{}\underline{11}\;\big|\;[y\mapsto{}\underline{13},x\mapsto{}\underline{11}]\;\big|\;[\underline{13}\mapsto{}2,\underline{12}\mapsto{}\langle{}(\lambda{}y.x),[x\mapsto{}\underline{11}]\rangle{},\underline{11}\mapsto{}1,\underline{10}\mapsto{}\langle{}(\lambda{}x.(\lambda{}y.x)),[]\rangle{},\underline{9}\mapsto{}3]\;\big|\;[]\big\rangle{})$

In [28]:
run cek = case step cek of Just cek' -> run cek'
                           Nothing   -> cek

runExp e = run (OpE e, [], [(9,Vint 3)], [])

In [29]:
LaTeX fst_1_2

runExp fst_1_2
display (LaTeX it)

$(((\lambda{}x.(\lambda{}y.x))\;1)\;2)$

(OpV 11,[("y",13),("x",11)],[(13,Vint 2),(12,Clos (Lam "y" (Var "x")) [("x",11)]),(11,Vint 1),(10,Clos (Lam "x" (Lam "y" (Var "x"))) []),(9,Vint 3)],[])

$\big\langle{}\underline{11}\;\big|\;[y\mapsto{}\underline{13},x\mapsto{}\underline{11}]\;\big|\;[\underline{13}\mapsto{}2,\underline{12}\mapsto{}\langle{}(\lambda{}y.x),[x\mapsto{}\underline{11}]\rangle{},\underline{11}\mapsto{}1,\underline{10}\mapsto{}\langle{}(\lambda{}x.(\lambda{}y.x)),[]\rangle{},\underline{9}\mapsto{}3]\;\big|\;[]\big\rangle{}$